# Continuity Care Assistant (CCA)

**An AI-powered "Don't Miss" alert system for primary care physicians**

Author: J.D. Laurence-Chasen

---

### The Problem

Primary care physicians face an impossible information synthesis challenge:

- **<15 minutes** per appointment of face-to-face time with patients
- **100+ pages** of notes, labs, and imaging per patient
- **Critical patterns get missed**: slow-building disease presentations, imaging follow-ups, drug-disease interactions

> *"There is just so much data in [a patient's] chart. There's no way to review all of it before an appointment--important connections can be missed"*  
> — M.L.W, M.D., University of Washington, Interview

### Our Solution

CCA uses **MedGemma 1.5 4B** in a two-stage, patient-centered pipeline that ingests and synthesizes all of a patient's data, generating "Don't Miss" alerts that can be integrated directly into an EMR Dashboard for easy access before or during an appointment. CCA ensures that critical patterns won't be missed.

```
┌─────────────────────────────────────────────────────────────────┐
│              CCA: Continuity Care Assistant Pipeline            │
├─────────────────────────────────────────────────────────────────┤
│  ┌───────────────────────────────────────────────────────────┐  │
│  │                   STAGE 1: EXTRACTION                     │  │
│  │                     MedGemma 1.5 4B                       │  │
│  ├───────────────────────────────────────────────────────────|  │
│  │                                                           │  │
│  │  Clinical Notes ──→ symptoms, findings, unlisted meds     │  │
│  │  Chest X-rays   ──→ nodules, masses, abnormalities        │  │
│  │  CT/MRI Scans   ──→ 3D volume interpretation              │  │
│  │  Lab Reports    ──→ structured values, trends             │  │
│  │                                                           │  │
│  └───────────────────────────────────────────────────────────┘  │
│                               ▼                                 │
│                    ┌─────────────────────┐                      │
│                    │  Aggregated Context │                      │
│                    └─────────────────────┘                      │
│                               ▼                                 │
│  ┌───────────────────────────────────────────────────────────┐  │
│  │                   STAGE 2: SYNTHESIS                      │  │
│  │                   MedGemma 1.5 4B                         │  │
│  ├───────────────────────────────────────────────────────────┤  │
│  │                                                           │  │
│  │  Cross-modal reasoning across all extracted findings      │  │
│  │  Pattern detection across longitudinal visits             │  │
│  │  Risk factor correlation and amplification                |  |
|  |  Safety rules integration (NSAID+CKD, etc.)               |  |
│  │  Alert generation with evidence chains and recommendations│  │
│  │                                                           │  │
│  └───────────────────────────────────────────────────────────┘  │
│                               ▼                                 │
│                    ┌─────────────────────┐                      │
│                    │ "Don't Miss" Alerts │                      │
│                    └─────────────────────┘                      │
└─────────────────────────────────────────────────────────────────┘
```

---
## 1. Setup

In [ ]:
# Install dependencies (uncomment on first run)
#!pip install transformers torch accelerate bitsandbytes

from dataclasses import dataclass, field
from datetime import datetime, timedelta
from enum import Enum
from typing import Optional
from IPython.display import HTML, display
import json
import re

# Set to True if running on GPU (Colab A100, etc.)
USE_REAL_MODELS = False
USE_MEDGEMMA = False  

print("✓ Setup complete")

---
## 2. Data Models

Simple dataclasses to represent patient data and alerts.

In [ ]:
class AlertSeverity(Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"

@dataclass
class Lab:
    name: str
    value: float
    unit: str
    date: datetime
    ref_low: Optional[float] = None
    ref_high: Optional[float] = None
    
    @property
    def is_abnormal(self) -> bool:
        if self.ref_low and self.value < self.ref_low:
            return True
        if self.ref_high and self.value > self.ref_high:
            return True
        return False

@dataclass
class Medication:
    name: str
    dose: str
    start_date: datetime
    prescriber: str

@dataclass
class Imaging:
    modality: str
    date: datetime
    indication: str
    findings: str
    impression: str
    follow_up: Optional[str] = None  # e.g., "CT chest in 3 months"
    follow_up_completed: bool = False

@dataclass
class Note:
    date: datetime
    author: str
    note_type: str
    chief_complaint: str
    content: str

@dataclass
class Patient:
    id: str
    name: str
    age: int
    sex: str
    problems: list[str] = field(default_factory=list)
    medications: list[Medication] = field(default_factory=list)
    labs: list[Lab] = field(default_factory=list)
    imaging: list[Imaging] = field(default_factory=list)
    notes: list[Note] = field(default_factory=list)
    smoking_status: str = "never"
    family_history: list[str] = field(default_factory=list)
    
    def get_lab_trend(self, lab_name: str) -> list[Lab]:
        """Get all results for a lab test, sorted by date."""
        return sorted(
            [l for l in self.labs if l.name == lab_name],
            key=lambda x: x.date
        )
    
    def has_medication_class(self, drug_class: list[str]) -> Optional[Medication]:
        """Check if patient is on any medication in the given class."""
        for med in self.medications:
            if any(drug.lower() in med.name.lower() for drug in drug_class):
                return med
        return None

@dataclass
class Alert:
    severity: AlertSeverity
    headline: str
    summary: str
    evidence: list[str]
    recommendations: list[str]

print("✓ Data models defined")

---
## 3. Synthetic Patients

Three patients demonstrating different alert types, **sourced from real interviews with practicing primary care physicians**:

| Patient | Age | Alert Type | Clinical Scenario |
|---------|-----|------------|-------------------|
| Maria Klein | 58 | Slow-building pattern | Progressive iron deficiency anemia + weight loss + GI symptoms |
| James Morrison | 64 | Missed follow-up | Lung nodule requiring follow-up CT, never completed |
| Dorothy Williams | 72 | Drug-disease interaction | NSAID prescribed despite CKD diagnosis |

In [ ]:
TODAY = datetime(2026, 2, 5)

# =============================================================================
# PATIENT 1: Maria Klein - "The Slow-Building Pattern"
# =============================================================================
maria = Patient(
    id="MK-001",
    name="Maria Klein",
    age=58,
    sex="Female",
    problems=["Hypertension", "Hyperlipidemia", "GERD"],
    smoking_status="never",
    family_history=["Mother: breast cancer at 65", "Father: MI at 70"],
    medications=[
        Medication("Lisinopril", "10mg daily", TODAY - timedelta(days=365), "Dr. Smith"),
        Medication("Atorvastatin", "20mg daily", TODAY - timedelta(days=365), "Dr. Smith"),
        Medication("Omeprazole", "20mg daily", TODAY - timedelta(days=180), "Dr. Smith"),
    ],
    labs=[
        # Hemoglobin trend - subtle decline
        Lab("Hemoglobin", 13.2, "g/dL", TODAY - timedelta(days=240), 12.0, 16.0),
        Lab("Hemoglobin", 12.8, "g/dL", TODAY - timedelta(days=180), 12.0, 16.0),
        Lab("Hemoglobin", 12.1, "g/dL", TODAY - timedelta(days=90), 12.0, 16.0),
        Lab("Hemoglobin", 11.4, "g/dL", TODAY - timedelta(days=14), 12.0, 16.0),
        # MCV trend - becoming microcytic
        Lab("MCV", 88, "fL", TODAY - timedelta(days=240), 80, 100),
        Lab("MCV", 82, "fL", TODAY - timedelta(days=90), 80, 100),
        Lab("MCV", 76, "fL", TODAY - timedelta(days=14), 80, 100),
        # Iron studies
        Lab("Ferritin", 18, "ng/mL", TODAY - timedelta(days=14), 20, 200),
        Lab("Iron", 35, "mcg/dL", TODAY - timedelta(days=14), 60, 170),
        # Stool guaiac
        Lab("Stool Guaiac", 1, "pos", TODAY - timedelta(days=7), None, None),  # 1 = positive
    ],
    notes=[
        Note(
            date=TODAY - timedelta(days=240),
            author="Dr. Smith",
            note_type="Annual Physical",
            chief_complaint="Annual exam",
            content="""58yo F presents for annual physical. Reports feeling 'a bit more tired than usual' 
over past few weeks, attributes to stress at work. Denies chest pain, SOB, palpitations. 
Weight 145 lbs (stable). BP 128/82. Labs ordered including CBC, CMP, lipids. 
Due for colonoscopy - discussed, patient wants to wait until after busy season at work."""
        ),
        Note(
            date=TODAY - timedelta(days=120),
            author="Dr. Smith",
            note_type="Office Visit",
            chief_complaint="Fatigue, unintentional weight loss",
            content="""Patient returns reporting persistent fatigue despite improved sleep. 
Notes clothes fitting looser - reports 8 lb weight loss over 4 months without trying.
Appetite slightly decreased. Occasional bloating after meals. Denies N/V, melena, hematochezia.
Weight today 137 lbs (down from 145). Discussed colonoscopy again - scheduled for next month."""
        ),
        Note(
            date=TODAY - timedelta(days=14),
            author="Dr. Smith",
            note_type="Office Visit",
            chief_complaint="Constipation, bloating",
            content="""Reports worsening constipation over past 3 weeks, alternating with loose stools.
Persistent bloating and early satiety. Continued weight loss - now 132 lbs.
Labs show Hgb 11.4 (was 12.8 six months ago), MCV 76, Ferritin 18.
Stool guaiac positive. Iron deficiency anemia with GI symptoms - colonoscopy was cancelled 
by patient due to scheduling conflict. Need to reschedule urgently."""
        ),
    ],
)

# =============================================================================
# PATIENT 2: James Morrison - "The Missed Follow-up"
# =============================================================================
james = Patient(
    id="JM-002",
    name="James Morrison",
    age=64,
    sex="Male",
    problems=["Hypertension", "Type 2 Diabetes", "Former smoker"],
    smoking_status="former (20 pack-years, quit 5 years ago)",
    family_history=["Father: lung cancer, died at 62", "Mother: COPD"],
    medications=[
        Medication("Metformin", "1000mg BID", TODAY - timedelta(days=730), "Dr. Johnson"),
        Medication("Amlodipine", "5mg daily", TODAY - timedelta(days=365), "Dr. Johnson"),
    ],
    labs=[
        Lab("Calcium", 10.2, "mg/dL", TODAY - timedelta(days=180), 8.5, 10.5),
        Lab("Calcium", 10.8, "mg/dL", TODAY - timedelta(days=14), 8.5, 10.5),
        Lab("PTH", 18, "pg/mL", TODAY - timedelta(days=14), 15, 65),  # Low-normal (suppressed)
        Lab("HbA1c", 7.2, "%", TODAY - timedelta(days=90), None, 7.0),
    ],
    imaging=[
        Imaging(
            modality="Chest X-ray",
            date=TODAY - timedelta(days=130),
            indication="Pre-operative clearance for knee arthroscopy",
            findings="8mm nodule in right upper lobe. No prior imaging for comparison. Lungs otherwise clear.",
            impression="8mm RUL pulmonary nodule. Given smoking history, recommend CT chest in 3 months for further evaluation per Fleischner criteria.",
            follow_up="CT chest in 3 months",
            follow_up_completed=False,
        ),
    ],
    notes=[
        Note(
            date=TODAY - timedelta(days=130),
            author="Dr. Johnson",
            note_type="Pre-op Visit",
            chief_complaint="Pre-op clearance",
            content="""64yo M with HTN, T2DM, former smoker here for pre-op clearance for knee arthroscopy.
CXR shows 8mm RUL nodule - new finding, no prior imaging. Given smoking history (20 pack-years)
and family history of lung cancer, this requires follow-up CT in 3 months per Fleischner.
Discussed with patient - he understands need for follow-up. Will order CT for 3 months out.
Cleared for surgery from cardiac standpoint."""
        ),
        Note(
            date=TODAY - timedelta(days=45),
            author="Dr. Patel",
            note_type="Urgent Care",
            chief_complaint="Upper respiratory infection",
            content="""Patient presents with 5 days of cough, congestion, low-grade fever.
No SOB, chest pain. Lungs with scattered rhonchi, no wheezes. Likely viral URI.
Supportive care recommended. Return if worsening."""
        ),
        Note(
            date=TODAY - timedelta(days=14),
            author="Dr. Johnson",
            note_type="Office Visit",
            chief_complaint="Routine diabetes follow-up",
            content="""Here for DM follow-up. A1c 7.2% - slightly above goal. Increase metformin?
Incidentally noted calcium slightly elevated at 10.8. PTH 18 (low-normal).
Will monitor calcium. Continue current regimen for now."""
        ),
    ],
)

# =============================================================================
# PATIENT 3: Dorothy Williams - "The Drug-Disease Interaction"
# =============================================================================
dorothy = Patient(
    id="DW-003",
    name="Dorothy Williams",
    age=72,
    sex="Female",
    problems=["CKD Stage 3a", "Hypertension", "Osteoarthritis", "Type 2 Diabetes"],
    smoking_status="never",
    family_history=["Brother: ESRD on dialysis"],
    medications=[
        Medication("Losartan", "50mg daily", TODAY - timedelta(days=730), "Dr. Chen"),
        Medication("Metformin", "500mg BID", TODAY - timedelta(days=730), "Dr. Chen"),
        Medication("Acetaminophen", "650mg TID PRN", TODAY - timedelta(days=365), "Dr. Chen"),
        Medication("Meloxicam", "15mg daily", TODAY - timedelta(days=18), "Dr. Roberts (Ortho)"),  # NEW!
    ],
    labs=[
        # Creatinine trend - stable until recently
        Lab("Creatinine", 1.3, "mg/dL", TODAY - timedelta(days=365), 0.6, 1.2),
        Lab("Creatinine", 1.3, "mg/dL", TODAY - timedelta(days=180), 0.6, 1.2),
        Lab("Creatinine", 1.4, "mg/dL", TODAY - timedelta(days=90), 0.6, 1.2),
        Lab("Creatinine", 1.8, "mg/dL", TODAY - timedelta(days=7), 0.6, 1.2),  # Acute rise!
        # eGFR trend
        Lab("eGFR", 52, "mL/min", TODAY - timedelta(days=365), 60, None),
        Lab("eGFR", 51, "mL/min", TODAY - timedelta(days=180), 60, None),
        Lab("eGFR", 48, "mL/min", TODAY - timedelta(days=90), 60, None),
        Lab("eGFR", 35, "mL/min", TODAY - timedelta(days=7), 60, None),  # Significant drop!
    ],
    notes=[
        Note(
            date=TODAY - timedelta(days=365),
            author="Dr. Chen",
            note_type="Annual Visit",
            chief_complaint="Annual exam",
            content="""72yo F with CKD 3a, HTN, T2DM, OA for annual exam. CKD stable - eGFR 52.
IMPORTANT: Avoid NSAIDs due to CKD - discussed with patient. Continue acetaminophen PRN for OA pain.
BP well controlled on losartan. A1c 6.8 - at goal."""
        ),
        Note(
            date=TODAY - timedelta(days=21),
            author="Dr. Roberts",
            note_type="Orthopedics Consult",
            chief_complaint="Knee pain",
            content="""72yo F referred for right knee pain. X-ray shows moderate OA with joint space narrowing.
Conservative management recommended. Starting meloxicam 15mg daily for inflammation.
Physical therapy referral placed. Follow-up in 6 weeks."""
        ),
        Note(
            date=TODAY - timedelta(days=7),
            author="Dr. Chen",
            note_type="Lab Review",
            chief_complaint="Lab follow-up",
            content="""Routine CKD monitoring labs. Creatinine 1.8 (was 1.4 three months ago) - 
concerning rise. eGFR dropped to 35 from 48. Patient reports knee feeling better.
Need to review medication list for nephrotoxins."""
        ),
    ],
)

PATIENTS = {"maria": maria, "james": james, "dorothy": dorothy}
print(f"✓ Loaded {len(PATIENTS)} synthetic patients")

---
## 4. Display Helpers

In [ ]:
def show_patient(patient: Patient):
    """Display patient summary card."""
    problems = ", ".join(patient.problems[:4])
    meds = ", ".join(m.name.split()[0] for m in patient.medications[:4])
    html = f"""
    <div style="background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%); 
                border-radius: 10px; padding: 20px; color: white; margin: 15px 0; font-family: system-ui;">
        <div style="display: flex; justify-content: space-between; align-items: start;">
            <div>
                <h2 style="margin: 0; font-size: 24px;">{patient.name}</h2>
                <p style="margin: 5px 0; opacity: 0.8;">{patient.age}yo {patient.sex} · MRN: {patient.id}</p>
            </div>
            <div style="text-align: right; font-size: 13px; opacity: 0.7;">
                <div>🚬 {patient.smoking_status}</div>
            </div>
        </div>
        <hr style="border: none; border-top: 1px solid rgba(255,255,255,0.2); margin: 15px 0;">
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px; font-size: 14px;">
            <div><strong>Problems:</strong> {problems}</div>
            <div><strong>Medications:</strong> {meds}</div>
        </div>
    </div>
    """
    display(HTML(html))

def show_lab_trend(patient: Patient, lab_name: str):
    """Display a lab trend as ASCII chart."""
    labs = patient.get_lab_trend(lab_name)
    if not labs:
        print(f"No {lab_name} results found")
        return
    
    print(f"\n📊 {lab_name} Trend")
    print("─" * 55)
    
    values = [l.value for l in labs]
    min_val, max_val = min(values), max(values)
    range_val = max_val - min_val if max_val != min_val else 1
    
    for lab in labs:
        date_str = lab.date.strftime("%Y-%m-%d")
        flag = "⚠️" if lab.is_abnormal else "✓ "
        bar_len = int(((lab.value - min_val) / range_val) * 20) + 1
        bar = "█" * bar_len
        ref = ""
        if lab.ref_low and lab.ref_high:
            ref = f"(ref: {lab.ref_low}-{lab.ref_high})"
        print(f"{date_str} │ {lab.value:6.1f} {lab.unit:8} {flag} {bar} {ref}")

def show_alert(alert: Alert):
    """Display an alert card."""
    colors = {
        AlertSeverity.CRITICAL: ("#dc3545", "🔴"),
        AlertSeverity.HIGH: ("#fd7e14", "🟠"),
        AlertSeverity.MEDIUM: ("#ffc107", "🟡"),
    }
    color, icon = colors.get(alert.severity, ("#6c757d", "⚪"))
    
    evidence_html = "\n".join(f"<li>{e}</li>" for e in alert.evidence)
    recs_html = "\n".join(f"<li><strong>{r}</strong></li>" for r in alert.recommendations)
    
    html = f"""
    <div style="border-left: 4px solid {color}; background: #fff; border-radius: 8px; 
                padding: 16px; margin: 15px 0; box-shadow: 0 2px 4px rgba(0,0,0,0.1); font-family: system-ui; max-width: 500px;">
        <div style="display: flex; align-items: center; gap: 10px; margin-bottom: 10px;">
            <span style="font-size: 20px;">{icon}</span>
            <span style="background: {color}; color: white; padding: 3px 10px; border-radius: 4px; 
                        font-size: 12px; font-weight: 600; text-transform: uppercase;">Don't Miss: {alert.severity.value}</span>
        </div>
        <h3 style="margin: 0 0 12px 0; color: #1a1a1a;">{alert.headline}</h3>
        
        <details style="margin-bottom: 12px;">
            <summary style="cursor: pointer; font-weight: 600; color: #333;">📋 Evidence ({len(alert.evidence)} items)</summary>
            <ul style="margin-top: 8px; color: #666; line-height: 1.6;">{evidence_html}</ul>
        </details>
        
        <details style="margin-bottom: 0;">
            <summary style="cursor: pointer; font-weight: 600; color: #0066cc;">⚡ Recommended Actions ({len(alert.recommendations)})</summary>
            <div style="background: #f0f7ff; padding: 12px; border-radius: 6px; margin-top: 8px;">
                <ul style="margin: 0; padding-left: 20px; color: #333;">{recs_html}</ul>
            </div>
        </details>
    </div>
    """
    display(HTML(html))

print("✓ Display helpers ready")

---
## 5. Stage 1: Data Extraction

MedGemma 1.5 4B handles all extraction tasks in Stage 1:

- **Clinical notes** → symptoms, findings, follow-up items
- **Medical images** → abnormalities, measurements, follow-up recommendations  
- **Lab reports** → structured values (supplemented by deterministic trend analysis)
- Note that, for the purposes of this demo, PDFs of notes/charts are not used, but could easily be added in the future.

### 5a. Clinical Note Extraction

In [ ]:
# =============================================================================
# STAGE 1a: Clinical Note Extraction (MedGemma 1.5 4B)
# =============================================================================

NOTE_EXTRACTION_PROMPT = """Extract structured clinical information from this note.

NOTE:
{note_text}

Return JSON with:
{{
  "symptoms": [
    {{"symptom": "name", "duration": "timeframe", "severity": "mild/moderate/severe", "trend": "stable/worsening/improving"}}
  ],
  "clinical_findings": [
    {{"finding": "name", "value": "measurement if applicable", "interpretation": "normal/abnormal/concerning"}}
  ],
  "follow_up_items": [
    {{"item": "description", "timeframe": "when", "completed": true/false}}
  ],
  "red_flags": ["any alarming findings"]
}}

Be specific and extract actual values from the note."""

@dataclass
class ExtractedNote:
    """Structured extraction from a clinical note."""
    date: datetime
    note_type: str
    symptoms: list[dict]
    findings: list[dict]
    follow_ups: list[dict]
    red_flags: list[str]

class NoteExtractor:
    """
    MedGemma 1.5 4B for clinical note extraction.
    
    MedGemma 1.5 4B specifically "delivers improved accuracy on medical text reasoning" 
    and "expands support for extraction of structured data from unstructured medical 
    lab reports and EHR understanding." (from MedGemma documentation)
    """
    
    def __init__(self, use_real_model: bool = False):
        self.use_real_model = use_real_model
        if use_real_model:
            from transformers import AutoProcessor, AutoModelForImageTextToText
            import torch
            print("Loading MedGemma 1.5 4B for note extraction...")
            self.processor = AutoProcessor.from_pretrained("google/medgemma-1.5-4b-it")
            self.model = AutoModelForImageTextToText.from_pretrained(
                "google/medgemma-1.5-4b-it",
                torch_dtype=torch.bfloat16,
                device_map="auto",
            )
    
    def extract(self, note: Note) -> ExtractedNote:
        """Extract structured information from a clinical note."""
        if self.use_real_model:
            return self._extract_with_model(note)
        else:
            return self._mock_extract(note)
    
    def _extract_with_model(self, note: Note) -> ExtractedNote:
        import torch
        prompt = NOTE_EXTRACTION_PROMPT.format(note_text=note.content)
        
        # Use proper MedGemma chat template format
        messages = [
            {
                "role": "user",
                "content": [{"type": "text", "text": prompt}]
            }
        ]
        
        inputs = self.processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(self.model.device, dtype=torch.bfloat16)
        
        input_len = inputs["input_ids"].shape[-1]
        
        with torch.inference_mode():
            generation = self.model.generate(**inputs, max_new_tokens=800, do_sample=False)
            generation = generation[0][input_len:]
        
        response = self.processor.decode(generation, skip_special_tokens=True)
        
        # Parse JSON from response
        data = self._parse_json(response)
        return ExtractedNote(
            date=note.date,
            note_type=note.note_type,
            symptoms=data.get("symptoms", []),
            findings=data.get("clinical_findings", []),
            follow_ups=data.get("follow_up_items", []),
            red_flags=data.get("red_flags", []),
        )
    
    def _mock_extract(self, note: Note) -> ExtractedNote:
        """Mock extraction based on keyword analysis."""
        content = note.content.lower()
        
        symptoms = []
        if "fatigue" in content or "tired" in content:
            symptoms.append({"symptom": "fatigue", "duration": "weeks", "severity": "moderate", "trend": "persistent"})
        if "weight loss" in content:
            symptoms.append({"symptom": "unintentional weight loss", "duration": "months", "severity": "concerning", "trend": "progressive"})
        if "constipation" in content:
            symptoms.append({"symptom": "constipation", "duration": "weeks", "severity": "moderate", "trend": "worsening"})
        if "bloating" in content:
            symptoms.append({"symptom": "bloating", "duration": "weeks", "severity": "moderate", "trend": "worsening"})
        if "cough" in content:
            symptoms.append({"symptom": "cough", "duration": "days", "severity": "mild", "trend": "stable"})
        if "knee pain" in content or "pain" in content:
            symptoms.append({"symptom": "joint pain", "duration": "chronic", "severity": "moderate", "trend": "stable"})
        
        findings = []
        if "nodule" in content:
            findings.append({"finding": "pulmonary nodule", "value": "8mm RUL", "interpretation": "requires follow-up"})
        if "hgb" in content or "hemoglobin" in content:
            findings.append({"finding": "anemia", "value": "Hgb declining", "interpretation": "concerning"})
        if "guaiac" in content and "positive" in content:
            findings.append({"finding": "stool guaiac", "value": "positive", "interpretation": "GI blood loss"})
        if "creatinine" in content and ("1.8" in content or "rise" in content or "drop" in content):
            findings.append({"finding": "acute kidney injury", "value": "Cr rising", "interpretation": "concerning"})
        
        follow_ups = []
        if "colonoscopy" in content:
            completed = "cancelled" not in content and "deferred" not in content and "wait" not in content
            follow_ups.append({"item": "colonoscopy", "timeframe": "scheduled", "completed": completed})
        if "ct" in content and "3 month" in content:
            follow_ups.append({"item": "CT chest follow-up", "timeframe": "3 months", "completed": False})
        if "follow-up" in content or "f/u" in content:
            follow_ups.append({"item": "imaging follow-up recommended", "timeframe": "per note", "completed": False})
        
        red_flags = []
        if "weight loss" in content and ("fatigue" in content or "anemia" in content):
            red_flags.append("Constitutional symptoms with anemia - malignancy workup needed")
        if "nodule" in content and ("smok" in content or "family" in content):
            red_flags.append("Lung nodule in high-risk patient")
        if "nsaid" in content.lower() or "meloxicam" in content.lower():
            if "ckd" in content.lower() or "kidney" in content.lower():
                red_flags.append("NSAID prescribed to patient with renal disease")
        
        return ExtractedNote(
            date=note.date,
            note_type=note.note_type,
            symptoms=symptoms,
            findings=findings,
            follow_ups=follow_ups,
            red_flags=red_flags,
        )
    
    def _parse_json(self, text: str) -> dict:
        match = re.search(r'\{[\s\S]*\}', text)
        if match:
            try:
                return json.loads(match.group())
            except json.JSONDecodeError:
                pass
        return {}

print("✓ NoteExtractor ready (MedGemma 1.5 4B)")

### 5b. Medical Image Analysis

Note that, for the purposes of this demo, just chest x-rays are used, but MedGemma 1.5 4B supports a wide range of medical imaging modalities:
- **Chest X-rays** with longitudinal comparison
- **CT and MRI** volumes
- **Histopathology** whole-slide images

In [ ]:
# =============================================================================
# STAGE 1b: Medical Image Analysis (MedGemma 1.5 4B)
# =============================================================================

IMAGE_ANALYSIS_PROMPT = """Analyze this medical image and extract structured findings.

IMAGE METADATA:
- Modality: {modality}
- Indication: {indication}

Return JSON with:
{{
  "findings": [
    {{"finding": "name", "location": "anatomical location", "size": "if applicable", "confidence": "high/medium/low"}}
  ],
  "impression": "overall interpretation",
  "follow_up_needed": true/false,
  "follow_up_recommendation": "specific recommendation if needed",
  "urgency": "routine/soon/urgent"
}}

Be specific about locations and measurements."""

@dataclass
class ImageAnalysis:
    """Structured analysis from a medical image."""
    modality: str
    date: datetime
    findings: list[dict]  # {"finding": str, "location": str, "confidence": float}
    follow_up_needed: bool = False
    urgency: str = "routine"  # routine, soon, urgent

class ImageAnalyzer:
    """
    MedGemma 1.5 4B for medical image analysis.
    
    MedGemma 1.5 4B supports:
    - Chest X-rays with longitudinal comparison
    - CT and MRI volume interpretation
    - Histopathology whole-slide images
    - Anatomical localization
    """
    
    def __init__(self, use_real_model: bool = False):
        self.use_real_model = use_real_model
        if use_real_model:
            from transformers import AutoProcessor, AutoModelForImageTextToText
            import torch
            print("Loading MedGemma 1.5 4B for image analysis...")
            self.processor = AutoProcessor.from_pretrained("google/medgemma-1.5-4b-it")
            self.model = AutoModelForImageTextToText.from_pretrained(
                "google/medgemma-1.5-4b-it",
                torch_dtype=torch.bfloat16,
                device_map="auto",
            )
    
    def analyze(self, imaging: Imaging) -> ImageAnalysis:
        """Analyze a medical image study."""
        if self.use_real_model:
            return self._analyze_with_model(imaging)
        else:
            return self._mock_analyze(imaging)
    
    def _analyze_with_model(self, imaging: Imaging, image=None) -> ImageAnalysis:
        """Real model inference with MedGemma 1.5 4B."""
        import torch
        from PIL import Image
        
        prompt = IMAGE_ANALYSIS_PROMPT.format(
            modality=imaging.modality,
            indication=imaging.indication,
        )
        
        # Build message content - include image if provided
        content = []
        if image is not None:
            content.append({"type": "image", "image": image})
        content.append({"type": "text", "text": prompt})
        
        messages = [
            {
                "role": "user",
                "content": content
            }
        ]
        
        inputs = self.processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(self.model.device, dtype=torch.bfloat16)
        
        input_len = inputs["input_ids"].shape[-1]
        
        with torch.inference_mode():
            generation = self.model.generate(**inputs, max_new_tokens=800, do_sample=False)
            generation = generation[0][input_len:]
        
        response = self.processor.decode(generation, skip_special_tokens=True)
        
        # Parse JSON from response
        data = self._parse_json(response)
        return ImageAnalysis(
            modality=imaging.modality,
            date=imaging.date,
            findings=data.get("findings", []),
            follow_up_needed=data.get("follow_up_needed", False),
            urgency=data.get("urgency", "routine"),
        )
    
    def _mock_analyze(self, imaging: Imaging) -> ImageAnalysis:
        """Mock analysis based on imaging metadata and findings text."""
        findings = []
        follow_up_needed = False
        urgency = "routine"
        
        # Analyze based on modality and findings text
        findings_text = (imaging.findings + " " + imaging.impression).lower()
        
        if imaging.modality.lower() in ["chest x-ray", "cxr", "chest xray"]:
            if "nodule" in findings_text:
                size_match = re.search(r'(\d+)\s*mm', findings_text)
                size = size_match.group(1) if size_match else "unknown"
                findings.append({
                    "finding": "pulmonary nodule",
                    "location": "RUL" if "rul" in findings_text or "right upper" in findings_text else "lung",
                    "size_mm": size,
                    "confidence": 0.94,
                    "fleischner_category": "high-risk" if int(size) >= 8 else "low-risk"
                })
                follow_up_needed = True
                urgency = "soon" if int(size) >= 6 else "routine"
            
            if "mass" in findings_text:
                findings.append({
                    "finding": "pulmonary mass",
                    "location": "lung",
                    "confidence": 0.91,
                })
                follow_up_needed = True
                urgency = "urgent"
            
            if "clear" in findings_text and not findings:
                findings.append({
                    "finding": "no acute abnormality",
                    "location": "chest",
                    "confidence": 0.92,
                })
        
        # Check if follow-up was recommended
        if imaging.follow_up:
            follow_up_needed = True
            if "urgent" in imaging.follow_up.lower():
                urgency = "urgent"
            elif "3 month" in imaging.follow_up.lower():
                urgency = "soon"
        
        return ImageAnalysis(
            modality=imaging.modality,
            date=imaging.date,
            findings=findings,
            follow_up_needed=follow_up_needed,
            urgency=urgency,
        )
    
    def _parse_json(self, text: str) -> dict:
        match = re.search(r'\{[\s\S]*\}', text)
        if match:
            try:
                return json.loads(match.group())
            except json.JSONDecodeError:
                pass
        return {}

print("✓ ImageAnalyzer ready (MedGemma 1.5 4B)")

### 5c. Lab Trend Analysis (Deterministic)

Labs are already structured data—no model needed. We compute trends, flags, and clinical significance.

In [ ]:
# =============================================================================
# STAGE 1c: Lab Trend Analysis (Deterministic)
# =============================================================================

@dataclass
class LabTrendAnalysis:
    """Analysis of laboratory trends."""
    lab_name: str
    values: list[tuple[datetime, float]]  # (date, value) pairs
    trend: str  # "stable", "increasing", "decreasing"
    is_abnormal: bool
    clinical_flag: Optional[str] = None  # e.g., "iron deficiency pattern"

def analyze_lab_trends(patient: Patient) -> list[LabTrendAnalysis]:
    """Analyze all lab trends for a patient."""
    analyses = []
    
    # Group labs by name
    lab_groups = {}
    for lab in patient.labs:
        if lab.name not in lab_groups:
            lab_groups[lab.name] = []
        lab_groups[lab.name].append(lab)
    
    for lab_name, labs in lab_groups.items():
        labs_sorted = sorted(labs, key=lambda x: x.date)
        values = [(l.date, l.value) for l in labs_sorted]
        
        # Determine trend
        if len(values) >= 2:
            first_val, last_val = values[0][1], values[-1][1]
            if all(values[i][1] >= values[i+1][1] for i in range(len(values)-1)):
                trend = "decreasing"
            elif all(values[i][1] <= values[i+1][1] for i in range(len(values)-1)):
                trend = "increasing"
            elif abs(last_val - first_val) / max(first_val, 0.01) > 0.15:
                trend = "increasing" if last_val > first_val else "decreasing"
            else:
                trend = "stable"
        else:
            trend = "insufficient data"
        
        # Check if abnormal
        is_abnormal = any(l.is_abnormal for l in labs_sorted)
        
        # Clinical flags
        clinical_flag = None
        if lab_name == "Hemoglobin" and trend == "decreasing":
            mcv_labs = [l for l in patient.labs if l.name == "MCV"]
            ferritin_labs = [l for l in patient.labs if l.name == "Ferritin"]
            if mcv_labs and mcv_labs[-1].value < 80:
                if ferritin_labs and ferritin_labs[-1].value < 30:
                    clinical_flag = "Iron deficiency anemia pattern"
                else:
                    clinical_flag = "Microcytic anemia - evaluate iron studies"
        
        if lab_name == "Creatinine" and trend == "increasing":
            values_only = [v[1] for v in values]
            if len(values_only) >= 2 and values_only[-1] > values_only[-2] * 1.2:
                clinical_flag = "Acute kidney injury (>20% rise)"
        
        if lab_name == "Calcium" and labs_sorted[-1].value > 10.5:
            pth_labs = [l for l in patient.labs if l.name == "PTH"]
            if pth_labs and pth_labs[-1].value < 25:
                clinical_flag = "Hypercalcemia with suppressed PTH - evaluate for malignancy"
        
        analyses.append(LabTrendAnalysis(
            lab_name=lab_name,
            values=values,
            trend=trend,
            is_abnormal=is_abnormal,
            clinical_flag=clinical_flag,
        ))
    
    return analyses

print("✓ Lab trend analyzer ready")

### 5d. Aggregation: Building the Clinical Context

Combine all extracted data into a structured context for MedGemma synthesis.

In [ ]:
# =============================================================================
# STAGE 1d: Aggregation - Building Clinical Context
# =============================================================================

@dataclass
class AggregatedContext:
    """Complete extracted context ready for MedGemma synthesis."""
    patient: Patient
    note_extractions: list[ExtractedNote]
    image_analyses: list[ImageAnalysis]
    lab_trends: list[LabTrendAnalysis]
    
    def to_synthesis_prompt(self) -> str:
        """Format aggregated context for MedGemma 1.5 4B synthesis."""
        
        # Patient demographics
        sections = [
            f"# Patient: {self.patient.name}",
            f"**Demographics:** {self.patient.age}yo {self.patient.sex}",
            f"**Problems:** {', '.join(self.patient.problems)}",
            f"**Medications:** {', '.join(f'{m.name} {m.dose}' for m in self.patient.medications)}",
            f"**Smoking:** {self.patient.smoking_status}",
            f"**Family History:** {', '.join(self.patient.family_history) or 'None'}",
        ]
        
        # Extracted note findings
        sections.append("\n## Extracted from Clinical Notes (MedGemma 1.5 4B)")
        for ext in sorted(self.note_extractions, key=lambda x: x.date):
            sections.append(f"\n### {ext.date.strftime('%Y-%m-%d')} - {ext.note_type}")
            if ext.symptoms:
                sections.append("**Symptoms:**")
                for s in ext.symptoms:
                    sections.append(f"  - {s['symptom']}: {s.get('severity', '')} severity, {s.get('trend', '')} ({s.get('duration', '')})")
            if ext.findings:
                sections.append("**Findings:**")
                for f in ext.findings:
                    sections.append(f"  - {f['finding']}: {f.get('value', '')} [{f.get('interpretation', '')}]")
            if ext.follow_ups:
                sections.append("**Follow-up Items:**")
                for f in ext.follow_ups:
                    status = "✓ completed" if f.get('completed') else "⚠️ NOT completed"
                    sections.append(f"  - {f['item']} ({f.get('timeframe', '')}) - {status}")
            if ext.red_flags:
                sections.append("**🚨 Red Flags:**")
                for rf in ext.red_flags:
                    sections.append(f"  - {rf}")
        
        # Image analysis results
        if self.image_analyses:
            sections.append("\n## Image Analysis (MedGemma 1.5 4B)")
            for img in self.image_analyses:
                sections.append(f"\n### {img.date.strftime('%Y-%m-%d')} - {img.modality}")
                for f in img.findings:
                    conf = f.get('confidence', 0) * 100
                    sections.append(f"  - {f['finding']} at {f.get('location', 'unspecified')} (confidence: {conf:.0f}%)")
                    if 'fleischner_category' in f:
                        sections.append(f"    Fleischner category: {f['fleischner_category']}")
                if img.follow_up_needed:
                    sections.append(f"  **Follow-up needed:** urgency = {img.urgency}")
        
        # Lab trends
        sections.append("\n## Lab Trend Analysis")
        for lt in self.lab_trends:
            flag = f" 🚨 {lt.clinical_flag}" if lt.clinical_flag else ""
            abnormal = " [ABNORMAL]" if lt.is_abnormal else ""
            values_str = " → ".join(f"{v[1]}" for v in lt.values[-4:])  # Last 4 values
            sections.append(f"- **{lt.lab_name}:** {values_str} (trend: {lt.trend}){abnormal}{flag}")
        
        return "\n".join(sections)

def run_stage1_extraction(patient: Patient, note_extractor: NoteExtractor, image_analyzer: ImageAnalyzer) -> AggregatedContext:
    """Run Stage 1: Extract and analyze all patient data with MedGemma 1.5 4B."""
    
    # Extract from notes
    note_extractions = [note_extractor.extract(note) for note in patient.notes]
    
    # Analyze images
    image_analyses = [image_analyzer.analyze(img) for img in patient.imaging]
    
    # Analyze lab trends (deterministic)
    lab_trends = analyze_lab_trends(patient)
    
    return AggregatedContext(
        patient=patient,
        note_extractions=note_extractions,
        image_analyses=image_analyses,
        lab_trends=lab_trends,
    )

print("✓ Aggregation pipeline ready")

In [ ]:
# Initialize Stage 1 extractors (all using MedGemma 1.5 4B)
note_extractor = NoteExtractor(use_real_model=USE_REAL_MODELS)
image_analyzer = ImageAnalyzer(use_real_model=USE_REAL_MODELS)

print("✓ Stage 1 pipeline initialized")
print(f"   • NoteExtractor: {'MedGemma 1.5 4B' if USE_REAL_MODELS else 'Mock'}")
print(f"   • ImageAnalyzer: {'MedGemma 1.5 4B' if USE_REAL_MODELS else 'Mock'}")
print(f"   • LabAnalyzer: Deterministic (structured data)")

---
## 5. Safety Rules (Deterministic)

A minimal set of **high-confidence safety rules** that catch clear-cut contraindications. These supplement MedGemma's analysis, and could be expanded, revised, or replaced with additional AI models in future versions. 

> **Why use any deterministic rules?** Drug-disease interactions like NSAID+CKD have near-100% precision and zero ambiguity. Running them deterministically ensures we never miss these critical safety issues, even if MedGemma were to hallucinate or miss them.

In [ ]:
# Safety rules: High-confidence contraindications only
NSAIDS = ["ibuprofen", "naproxen", "meloxicam", "diclofenac", "celecoxib", "ketorolac", "indomethacin"]

def check_nsaid_in_ckd(patient: Patient) -> Optional[Alert]:
    """
    CRITICAL SAFETY RULE: NSAID use in patients with CKD.
    
    This is a clear-cut contraindication with no ambiguity—NSAIDs cause 
    acute kidney injury in CKD patients. We use a deterministic rule here
    because precision is 100% and we cannot afford to miss it.
    """
    has_ckd = any("ckd" in p.lower() or "chronic kidney" in p.lower() for p in patient.problems)
    if not has_ckd:
        return None
    
    nsaid = patient.has_medication_class(NSAIDS)
    if not nsaid:
        return None
    
    # Check for worsening renal function (strengthens the alert)
    cr_trend = patient.get_lab_trend("Creatinine")
    worsening = ""
    if len(cr_trend) >= 2:
        recent, previous = cr_trend[-1].value, cr_trend[-2].value
        if recent > previous * 1.2:  # >20% rise
            worsening = f" Creatinine has risen from {previous} to {recent} mg/dL since starting NSAID."
    
    return Alert(
        severity=AlertSeverity.CRITICAL,
        headline="NSAID Contraindicated in CKD",
        summary=f"{nsaid.name} was started {(TODAY - nsaid.start_date).days} days ago by {nsaid.prescriber}. "
                f"Patient has documented CKD—NSAIDs can cause acute kidney injury.{worsening}",
        evidence=[
            f"Diagnosis: CKD on problem list",
            f"Medication: {nsaid.name} {nsaid.dose} started {nsaid.start_date.strftime('%Y-%m-%d')}",
            f"Prescriber: {nsaid.prescriber}",
            f"Current creatinine: {cr_trend[-1].value if cr_trend else 'N/A'} mg/dL",
        ],
        recommendations=[
            "Discontinue NSAID immediately",
            "Contact prescribing physician about alternatives",
            "Recheck creatinine in 3-5 days",
            "Consider acetaminophen or topical therapies",
        ],
    )

def run_safety_rules(patient: Patient) -> list[Alert]:
    """Run deterministic safety rules. Returns high-confidence alerts only."""
    alerts = []
    
    # Add more safety rules here as needed (e.g., metformin + contrast, K+ sparing diuretic + ACE-I)
    nsaid_alert = check_nsaid_in_ckd(patient)
    if nsaid_alert:
        alerts.append(nsaid_alert)
    
    return alerts

print("✓ Safety rules ready (NSAID+CKD)")

### Future Extension: Medication Reconciliation

MedGemma's note extraction could also cross-reference medications mentioned in clinical notes against the patient's formal medication list. This would flag discrepancies such as:

- **Undocumented medications**: OTC drugs, supplements, or prescriptions from outside providers mentioned by the patient but not in the EHR
- **Discontinued medications**: Drugs still on the active list that the patient reports stopping
- **Dosage mismatches**: Patient-reported doses that differ from what's prescribed

This reconciliation step would strengthen the safety rules by ensuring they operate on a complete, accurate medication picture.

---
## 7. Stage 2: MedGemma 1.5 4B Alert Synthesis

**MedGemma is the core of CCA.** The model receives a structured prompt with longitudinal patient data and returns "Don't Miss" alerts with evidence chains.

In [ ]:
# =============================================================================
# MedGemma 1.5 4B Alert Synthesis
# =============================================================================

ALERT_SYNTHESIS_PROMPT = """You are a clinical decision support system helping primary care physicians identify "Don't Miss" findings.

You have been provided with structured extractions from a patient's medical record, all processed by MedGemma 1.5 4B:
- Clinical notes → symptoms, findings, follow-up items
- Medical images → abnormalities, measurements
- Lab trends → computed deterministically from structured data

Your task: Synthesize these findings to identify concerning patterns that a busy physician might miss.

{aggregated_context}

---

## Focus Areas
1. **Slow-building patterns** - Gradual changes across multiple visits suggesting serious disease
2. **Missed follow-ups** - Recommended tests/imaging that were never completed  
3. **Cross-modal connections** - Links between labs, imaging, symptoms that together suggest a diagnosis
4. **Risk factor amplification** - Findings more concerning given patient's specific risk factors

Return your analysis as JSON:
```json
{{
  "alerts": [
    {{
      "severity": "critical|high|medium",
      "headline": "Brief alert title",
      "summary": "2-3 sentence clinical explanation",
      "evidence": ["Specific finding 1", "Specific finding 2"],
      "recommendations": ["Action 1", "Action 2"]
    }}
  ]
}}
```

Be specific. Reference actual values, dates, and model outputs from the aggregated context."""


class MedGemmaSynthesizer:
    """
    MedGemma 1.5 4B for alert synthesis.
    
    Uses the same model as Stage 1 extraction, but for a different task:
    synthesizing cross-modal patterns into actionable clinical alerts.
    """
    
    def __init__(self, use_real_model: bool = False):
        self.use_real_model = use_real_model
        if use_real_model:
            from transformers import AutoProcessor, AutoModelForImageTextToText
            import torch
            print("Loading MedGemma 1.5 4B for synthesis...")
            self.processor = AutoProcessor.from_pretrained("google/medgemma-1.5-4b-it")
            self.model = AutoModelForImageTextToText.from_pretrained(
                "google/medgemma-1.5-4b-it",
                torch_dtype=torch.bfloat16,
                device_map="auto",
            )
            print(f"✓ MedGemma 1.5 4B loaded on {self.model.device}")
    
    def generate_alerts(self, context: AggregatedContext) -> list[Alert]:
        """Generate alerts from aggregated context."""
        if self.use_real_model:
            return self._generate_with_model(context)
        else:
            return self._mock_generate(context.patient)
    
    def _generate_with_model(self, context: AggregatedContext) -> list[Alert]:
        """Real model inference."""
        import torch
        
        prompt = ALERT_SYNTHESIS_PROMPT.format(
            aggregated_context=context.to_synthesis_prompt()
        )
        
        # Use proper MedGemma chat template format
        messages = [
            {
                "role": "user",
                "content": [{"type": "text", "text": prompt}]
            }
        ]
        
        inputs = self.processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(self.model.device, dtype=torch.bfloat16)
        
        input_len = inputs["input_ids"].shape[-1]
        
        with torch.inference_mode():
            generation = self.model.generate(**inputs, max_new_tokens=1500, do_sample=False)
            generation = generation[0][input_len:]
        
        response = self.processor.decode(generation, skip_special_tokens=True)
        return self._parse_alerts(response)
    
    def _parse_alerts(self, response: str) -> list[Alert]:
        """Extract and parse alerts from model response."""
        match = re.search(r'\{[\s\S]*\}', response)
        if not match:
            return []
        
        try:
            data = json.loads(match.group())
            alerts = []
            for a in data.get("alerts", []):
                severity_map = {"critical": AlertSeverity.CRITICAL, "high": AlertSeverity.HIGH, "medium": AlertSeverity.MEDIUM}
                alerts.append(Alert(
                    severity=severity_map.get(a.get("severity", "medium").lower(), AlertSeverity.MEDIUM),
                    headline=a.get("headline", "Alert"),
                    summary=a.get("summary", ""),
                    evidence=a.get("evidence", []),
                    recommendations=a.get("recommendations", []),
                ))
            return alerts
        except (json.JSONDecodeError, KeyError):
            return []
    
    def _mock_generate(self, patient: Patient) -> list[Alert]:
        """Mock alerts for demo without GPU."""
        
        if patient.id == "MK-001":  # Maria Klein - slow-building anemia
            return [Alert(
                severity=AlertSeverity.CRITICAL,
                headline="Iron Deficiency Anemia with Alarm Features — Rule Out GI Malignancy",
                summary="MedGemma 1.5 4B identified progressive microcytic anemia over 8 months with "
                        "unintentional weight loss (13 lbs), new GI symptoms, and positive stool guaiac. "
                        "In a 58-year-old, this constellation strongly suggests occult GI blood loss.",
                evidence=[
                    "Note extraction: 'fatigue' (3 visits), 'weight loss' (2 visits), 'bloating' (recent)",
                    "Lab trends: Hemoglobin 13.2→11.4 g/dL (decreasing), MCV 88→76 fL (microcytic)",
                    "Lab flags: Iron deficiency anemia pattern detected",
                    "Follow-up tracking: Colonoscopy deferred multiple times",
                    "Stool guaiac: POSITIVE",
                ],
                recommendations=[
                    "Schedule colonoscopy URGENTLY — do not allow further deferral",
                    "GI referral for expedited workup",
                    "Consider CT abdomen/pelvis if colonoscopy delayed",
                    "Check CEA level",
                ],
            )]
        
        elif patient.id == "JM-002":  # James Morrison - missed lung nodule follow-up
            return [Alert(
                severity=AlertSeverity.CRITICAL,
                headline="Lung Nodule Follow-up Overdue — High-Risk Patient",
                summary="MedGemma 1.5 4B detected 8mm pulmonary nodule on chest X-ray 4 months ago. "
                        "Follow-up CT was never completed. New hypercalcemia with suppressed PTH "
                        "raises concern for malignancy in this high-risk patient.",
                evidence=[
                    "Image analysis: 8mm nodule at RUL (94% confidence, Fleischner high-risk)",
                    "Follow-up tracking: 'CT chest in 3 months' — NOT COMPLETED (40+ days overdue)",
                    "Risk factors: 20 pack-year smoking, father died of lung cancer at 62",
                    "Lab trends: Calcium 10.2→10.8 mg/dL (increasing)",
                    "Lab flags: Hypercalcemia with suppressed PTH — evaluate for malignancy",
                ],
                recommendations=[
                    "Order CT chest WITHOUT contrast URGENTLY",
                    "Contact patient directly — do not rely on mailed reminder",
                    "Pulmonology referral given risk factors",
                    "Complete hypercalcemia workup: Vitamin D, PTHrP, SPEP",
                ],
            )]
        
        elif patient.id == "DW-003":  # Dorothy Williams - NSAID+CKD
            return [Alert(
                severity=AlertSeverity.HIGH,
                headline="Accelerated CKD Progression — Evaluate for Nephrotoxic Cause",
                summary="MedGemma 1.5 4B identified that eGFR dropped from 48 to 35 mL/min over 3 months, "
                        "significantly faster than prior stable trajectory. Temporal correlation with "
                        "recent orthopedics visit suggests medication-induced injury.",
                evidence=[
                    "Lab trends: eGFR 52→51→48→35 mL/min (accelerated decline)",
                    "Lab flags: Acute kidney injury (>20% Cr rise)",
                    "Note extraction: Recent orthopedics consult with new prescriptions",
                    "Temporal correlation: Decline began after orthopedics visit",
                ],
                recommendations=[
                    "Review all medications for nephrotoxins",
                    "Hold any identified nephrotoxins immediately",
                    "Recheck creatinine in 1 week",
                    "Nephrology referral if no reversible cause identified",
                ],
            )]
        
        return []


# =============================================================================
# Initialize
# =============================================================================
synthesizer = MedGemmaSynthesizer(use_real_model=USE_MEDGEMMA)

if not USE_MEDGEMMA:
    print("✓ MedGemma 1.5 4B Synthesizer ready (mock mode)")
    print("  Set USE_MEDGEMMA=True for real model inference")

---
## Patient 1: Maria Klein
### "The Slow-Building Pattern"

Maria is a 58-year-old woman who has been seen multiple times over 8 months. Each visit seemed routine, but connecting the dots reveals a concerning pattern: progressive iron deficiency anemia with weight loss and GI symptoms—a presentation that warrants urgent evaluation for GI malignancy.

In [ ]:
show_patient(maria)

In [ ]:
# Lab trends tell the story
show_lab_trend(maria, "Hemoglobin")
show_lab_trend(maria, "MCV")
show_lab_trend(maria, "Ferritin")

In [ ]:
# === CCA FULL PIPELINE: Maria Klein ===
print("="*70)
print("CCA MULTI-MODEL PIPELINE")
print("="*70)

# ─────────────────────────────────────────────────────────────────────
# STAGE 1: Extraction & Analysis
# ─────────────────────────────────────────────────────────────────────
print("\n┌─ STAGE 1: Data Extraction ─────────────────────────────────────┐")

# 1a. Note extraction (MedGemma 1.5 4B)
print("\n  📝 MedGemma 1.5 4B: Extracting from clinical notes...")
context = run_stage1_extraction(maria, note_extractor, image_analyzer)

for ext in context.note_extractions:
    print(f"\n     {ext.date.strftime('%Y-%m-%d')} - {ext.note_type}:")
    if ext.symptoms:
        print(f"       Symptoms: {', '.join(s['symptom'] for s in ext.symptoms)}")
    if ext.findings:
        print(f"       Findings: {', '.join(f['finding'] for f in ext.findings)}")
    if ext.red_flags:
        print(f"       🚨 Red flags: {len(ext.red_flags)}")

# 1b. Lab analysis (deterministic)
print("\n  📊 Lab Trend Analysis:")
for lt in context.lab_trends:
    if lt.clinical_flag:
        print(f"       {lt.lab_name}: {lt.trend} 🚨 {lt.clinical_flag}")
    elif lt.is_abnormal:
        print(f"       {lt.lab_name}: {lt.trend} [ABNORMAL]")

print("\n└────────────────────────────────────────────────────────────────┘")

# ─────────────────────────────────────────────────────────────────────
# STAGE 2: Synthesis (MedGemma 1.5 4B)
# ─────────────────────────────────────────────────────────────────────
print("\n┌─ STAGE 2: MedGemma 1.5 4B Synthesis ──────────────────────────┐")

# Show aggregated context size
synthesis_prompt = context.to_synthesis_prompt()
print(f"\n  📋 Aggregated context: {len(synthesis_prompt)} characters")
print(f"     • {len(context.note_extractions)} notes processed")
print(f"     • {len(context.lab_trends)} lab trends analyzed")
print(f"     • {len(context.image_analyses)} images analyzed")

print("\n  🧠 MedGemma 1.5 4B: Synthesizing alerts...")
medgemma_alerts = synthesizer.generate_alerts(context)
print(f"     → Generated {len(medgemma_alerts)} alert(s)")

print("\n└────────────────────────────────────────────────────────────────┘")

# ─────────────────────────────────────────────────────────────────────
# Safety Rules
# ─────────────────────────────────────────────────────────────────────
print("\n⚡ Safety Rules: ", end="")
safety_alerts = run_safety_rules(maria)
print(f"{len(safety_alerts)} alert(s)")

# ─────────────────────────────────────────────────────────────────────
# Display Alerts
# ─────────────────────────────────────────────────────────────────────
all_alerts = medgemma_alerts + safety_alerts

print("\n" + "="*70)
print("DON'T MISS ALERTS")
print("="*70)
for alert in all_alerts:
    show_alert(alert)

---
## Patient 2: James Morrison
### "The Missed Follow-up"

James had an incidental lung nodule found on a pre-op chest X-ray 4+ months ago. The recommended 3-month follow-up CT was never done. He's now presenting with hypercalcemia with suppressed PTH—a pattern concerning for malignancy.

In [ ]:
show_patient(james)

In [ ]:
# Show the imaging study that needs follow-up
print("📷 Imaging History:")
print("-" * 60)
for img in james.imaging:
    days_ago = (TODAY - img.date).days
    status = "⚠️ NOT COMPLETED" if not img.follow_up_completed else "✓ Completed"
    print(f"{img.modality} ({days_ago} days ago)")
    print(f"   Finding: {img.findings}")
    print(f"   Recommended: {img.follow_up}")
    print(f"   Status: {status}")

In [ ]:
# === CCA FULL PIPELINE: James Morrison ===
print("="*70)
print("CCA MULTI-MODEL PIPELINE")
print("="*70)

# ─────────────────────────────────────────────────────────────────────
# STAGE 1: Extraction & Analysis (MedGemma 1.5 4B)
# ─────────────────────────────────────────────────────────────────────
print("\n┌─ STAGE 1: Data Extraction (MedGemma 1.5 4B) ─────────────────┐")

context = run_stage1_extraction(james, note_extractor, image_analyzer)

# 1a. Note extraction
print("\n  📝 Clinical note extraction...")
for ext in context.note_extractions:
    if ext.findings or ext.follow_ups:
        print(f"     {ext.date.strftime('%Y-%m-%d')}: ", end="")
        items = [f['finding'] for f in ext.findings] + [f['item'] for f in ext.follow_ups]
        print(", ".join(items[:3]))

# 1b. Image analysis - THIS IS KEY FOR JAMES
print("\n  🔬 Medical image analysis...")
for img_analysis in context.image_analyses:
    print(f"     {img_analysis.modality} ({img_analysis.date.strftime('%Y-%m-%d')}):")
    for finding in img_analysis.findings:
        conf = finding.get('confidence', 0) * 100
        print(f"       • {finding['finding']} at {finding.get('location', '?')} ({conf:.0f}% confidence)")
        if 'fleischner_category' in finding:
            print(f"         Fleischner: {finding['fleischner_category']}")
    if img_analysis.follow_up_needed:
        print(f"       ⚠️ Follow-up needed (urgency: {img_analysis.urgency})")

# 1c. Lab analysis
print("\n  📊 Lab Trend Analysis:")
for lt in context.lab_trends:
    if lt.clinical_flag:
        print(f"       {lt.lab_name}: 🚨 {lt.clinical_flag}")

print("\n└────────────────────────────────────────────────────────────────┘")

# ─────────────────────────────────────────────────────────────────────
# STAGE 2: Synthesis
# ─────────────────────────────────────────────────────────────────────
print("\n┌─ STAGE 2: MedGemma 1.5 4B Synthesis ──────────────────────────┐")

synthesis_prompt = context.to_synthesis_prompt()
print(f"\n  📋 Aggregated context: {len(synthesis_prompt)} characters")
print(f"     Includes: image findings + note extractions + lab trends")

print("\n  🧠 MedGemma 1.5 4B: Cross-modal reasoning...")
medgemma_alerts = synthesizer.generate_alerts(context)
print(f"     → Generated {len(medgemma_alerts)} alert(s)")

print("\n└────────────────────────────────────────────────────────────────┘")

# Safety Rules
print("\n⚡ Safety Rules: ", end="")
safety_alerts = run_safety_rules(james)
print(f"{len(safety_alerts)} alert(s)")

# Display
all_alerts = medgemma_alerts + safety_alerts

print("\n" + "="*70)
print("DON'T MISS ALERTS")
print("="*70)
for alert in all_alerts:
    show_alert(alert)

---
## Patient 3: Dorothy Williams
### "The Drug-Disease Interaction"

Dorothy has well-documented CKD Stage 3a. She was recently started on an NSAID by orthopedics for knee OA.

**This case demonstrates the hybrid approach**: The NSAID+CKD interaction is caught by a deterministic safety rule (100% precision), while MedGemma provides additional context about the accelerated renal decline.

In [ ]:
show_patient(dorothy)

In [ ]:
# Show the concerning renal function trend
show_lab_trend(dorothy, "Creatinine")
show_lab_trend(dorothy, "eGFR")

In [ ]:
# Show medication list highlighting the problem
print("\n💊 Current Medications:")
print("-" * 60)
for med in dorothy.medications:
    days_on = (TODAY - med.start_date).days
    flag = "⚠️ NSAID - CONTRAINDICATED IN CKD" if "meloxicam" in med.name.lower() else ""
    print(f"   {med.name} {med.dose}")
    print(f"      Started: {med.start_date.strftime('%Y-%m-%d')} ({days_on} days ago) by {med.prescriber}")
    if flag:
        print(f"      {flag}")

In [ ]:
# === CCA FULL PIPELINE: Dorothy Williams ===
print("="*70)
print("CCA MULTI-MODEL PIPELINE")
print("="*70)

# ─────────────────────────────────────────────────────────────────────
# STAGE 1: Extraction & Analysis
# ─────────────────────────────────────────────────────────────────────
print("\n┌─ STAGE 1: Data Extraction ─────────────────────────────────────┐")

context = run_stage1_extraction(dorothy, note_extractor, image_analyzer)

print("\n  📝 MedGemma 1.5 4B: Clinical note extraction...")
for ext in context.note_extractions:
    if ext.findings or ext.red_flags:
        print(f"     {ext.date.strftime('%Y-%m-%d')} ({ext.note_type}):")
        for f in ext.findings:
            print(f"       • {f['finding']}: {f.get('interpretation', '')}")
        for rf in ext.red_flags:
            print(f"       🚨 {rf}")

print("\n  📊 Lab Trend Analysis:")
for lt in context.lab_trends:
    if lt.clinical_flag:
        print(f"       {lt.lab_name}: 🚨 {lt.clinical_flag}")
    elif lt.is_abnormal:
        print(f"       {lt.lab_name}: {lt.trend} [ABNORMAL]")

print("\n└────────────────────────────────────────────────────────────────┘")

# ─────────────────────────────────────────────────────────────────────
# STAGE 2: Synthesis + Safety Rules
# ─────────────────────────────────────────────────────────────────────
print("\n┌─ STAGE 2: Synthesis + Safety ─────────────────────────────────┐")

print("\n  🧠 MedGemma 1.5 4B: Pattern synthesis...")
medgemma_alerts = synthesizer.generate_alerts(context)
print(f"     → Generated {len(medgemma_alerts)} alert(s)")

print("\n  ⚡ Safety Rules: Checking drug-disease interactions...")
safety_alerts = run_safety_rules(dorothy)
print(f"     → {len(safety_alerts)} CRITICAL safety alert(s)")
if safety_alerts:
    print(f"     → NSAID + CKD detected!")

print("\n└────────────────────────────────────────────────────────────────┘")

# Display: Safety alerts first (higher priority)
all_alerts = safety_alerts + medgemma_alerts

print("\n" + "="*70)
print("DON'T MISS ALERTS")
print("="*70)
for alert in all_alerts:
    show_alert(alert)